In [20]:
import gensim
import numpy as np
import pandas as pd

In [15]:
# Load pre-trained Word2Vec model.
model = gensim.models.Word2Vec.load("w2v_model/word2vec_WordNumBig2.model")

In [23]:
vector_size = model.wv.vector_size
vector_size

100

# 驗證集

In [83]:
val_df = pd.read_pickle("clean_Validation.pk")
text2vec = []
for text in val_df.clean_words:
    wordNumber = 0
    totalWordVec = np.zeros(vector_size)
    for word in text:
        try:
            totalWordVec += model.wv[word]
            wordNumber += 1
        except:
            continue
    text2vec.append(totalWordVec/wordNumber)
x_val = np.array(text2vec)

# 訓練集

In [84]:
test_df = pd.read_pickle("clean_Test_Intuitive_VAL.pk")

In [85]:
text2vec = []
for text in test_df.clean_words:
    wordNumber = 0
    totalWordVec = np.zeros(vector_size)
    for word in text:
        try:
            totalWordVec += model.wv[word]
            wordNumber += 1
        except:
            continue
    text2vec.append(totalWordVec/wordNumber)

X = np.array(text2vec)
y = test_df.y

# Random Forest

In [90]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=100, n_estimators = 200)
clf.fit(X, y)


pred = clf.predict(x_val)
sample = val_df
sample["Obesity"] = pred
sample = sample.drop(["clean_words"], axis=1)
sample.to_csv("sample_submission.csv", index=False)

# Naive Bayes

In [87]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
pred = gnb.fit(X, y).predict(x_val)
sample = val_df
sample["Obesity"] = pred
sample = sample.drop(["clean_words"], axis=1)
sample.to_csv("sample_submission.csv", index=False)

# XGboost

In [89]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X, y)

pred = xgb.predict(x_val)
sample = val_df
sample["Obesity"] = pred
sample = sample.drop(["clean_words"], axis=1)
sample.to_csv("sample_submission.csv", index=False)


[11:23:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
